In [ ]:
import numpy as np
import torch
from torch.optim import Adam
from FNO import FNO2d, get_dataloader
import matplotlib.pyplot as plt

# 1. load & preprocess data
# adjust paths as needed
X = np.load("../Fourier/navier/test_inputs.npy")[:,:,:,0,0,0]
Y = np.load("../Fourier/navier/test_label.npy")[:,:,:,0,0,0]


In [ ]:
# add channel dim
X = X[:, None, :, :]
Y = Y[:, None, :, :]

in_channel  = 16
out_channel = 16
modes1 = modes2 = 16
width  = 64

# replicate to match in/out channels
X = np.repeat(X, in_channel, axis=1)
Y = np.repeat(Y, out_channel, axis=1)


In [ ]:
# split train / test
train_input  = torch.tensor(X[:10000], dtype=torch.float32)
train_label  = torch.tensor(Y[:10000], dtype=torch.float32)
test_input   = torch.tensor(X[10000:11000], dtype=torch.float32)
test_label   = torch.tensor(Y[10000:11000], dtype=torch.float32)

# 2. dataloaders
batch_size = 100
train_loader = get_dataloader(train_input, train_label, batch_size, shuffle=True)
test_loader  = get_dataloader(test_input,  test_label,  batch_size, shuffle=False)

# 3. device, model, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FNO2d(in_channel, out_channel, modes1, modes2, width, torch.relu).to(device)
optimizer = Adam(model.parameters(), lr=3e-4)


In [ ]:
# 4. train
epochs = 100
history = model.train_model(train_loader, test_loader, optimizer, epochs, device)

In [ ]:
# 5. plot losses
plt.figure()
plt.plot(history['train_loss'], label='train')
plt.plot(history['val_loss'],   label='val')
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.legend()
plt.show()